## Установим зависимости

In [1]:
!pip install pip -U
!pip install torch==2.2.* torchvision==0.17
!pip install polygraphy==0.49.9
!pip install tensorrt==8.6.* --extra-index-url https://pypi.nvidia.com
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 22.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 28.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 24.7 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 33.4 MB/s eta 0:00:0031m32.9 MB/s eta 0:00:01
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled tr

## Подготовим полезные функции

In [2]:
from pathlib import Path

import torch

from torch.utils.data.dataloader import DataLoader 
from torchvision import transforms
from torchvision.datasets.imagenette import Imagenette

CLASSES_MAPPING = {
    0: 0,
    1: 217,
    2: 848,
    3: 491,
    4: 497,
    5: 566,
    6: 569,
    7: 571,
    8: 574,
    9: 701,
}

         
def imagenette_val_dataloader(batch_size, height, width):
    root_dir = Path("./imagenette")
    
    dataset = Imagenette(
        root=root_dir, 
        split="val", 
        download=not root_dir.exists(),
        transform=transforms.Compose([
            transforms.Resize((height, width)), 
            transforms.ToTensor(), 
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], inplace=True),
        ])
    )

    return DataLoader(dataset, batch_size=batch_size)


def validate(model, batch_size, height, width):
    val_dataloder = imagenette_val_dataloader(batch_size, height, width)

    with torch.no_grad():
        acc = []
        for images, labels in val_dataloder:
            output = model(images.cuda())
            _, predicted_labels = torch.max(output, dim=1)
            predicted_labels = predicted_labels.cpu().tolist()
            
            acc += [predicted_label == CLASSES_MAPPING[label] for predicted_label, label in zip(predicted_labels, labels.tolist())]
            
        print(f"acc = {sum(acc) * 100 / len(acc):.2f}%")

/home/hflabs/anaconda3/envs/ds/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Воспроизвидите функцию для замера latency из лекции (10 баллов)

In [9]:
import numpy as np
from time import perf_counter

def latency_benchmark(model, test_input, warmup_n=10, benchmark_n=100):
    # Warmup
    for _ in range(warmup_n):
        model(test_input)
    torch.cuda.synchronize()

    # Benchmark with synchronization
    stats = []
    for _ in range(benchmark_n):
        t0 = perf_counter()
        model(test_input)
        torch.cuda.synchronize()
        t1 = perf_counter()
        stats.append(t1 - t0)

    stats = np.array(stats) * 1000  # to ms
    mean_ms = stats.mean()
    std_ms = stats.std()

    print(f"{mean_ms:.3f}ms +- {std_ms:.3f}ms")

    assert (std_ms / mean_ms) < 0.1, \
        "слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков"


## Проверяем как работает функция замера latency

In [14]:
from torchvision.models import mobilenet_v2
from torchvision.models import MobileNetV2


model = mobilenet_v2(weights=MobileNetV2).eval().cuda()

# запускаем под no_grad, чтобы минимизировать потребление памяти (исключает выделение памяти под градиенты)
with torch.no_grad():
    latency_benchmark(
        model, 
        torch.ones(1, 3, 640, 480, device="cuda"), 
        warmup_n=10, 
        benchmark_n=10000,
    )

2.988ms +- 0.193ms


## Напишите функцию для записи CUDA graph (10 баллов)

Функция должна вернуть объект CUDAGraph с записанным графом, входной тензор для передачи данных и выходной тензор для копирования результатов.

In [20]:
import torch

def record_CUDA_graph(model, batch_size, height, width, warmup_n=10):
    assert torch.cuda.is_available(), "CUDA is not available"

    model = model.eval().cuda()
    stream = torch.cuda.Stream()

    # Заранее создаём input и output буферы
    static_input = torch.empty(batch_size, 3, height, width, device='cuda').contiguous()
    with torch.no_grad():
        static_output = model(static_input)
    static_output = static_output.contiguous()

    static_input.requires_grad_(False)
    static_output.requires_grad_(False)

    # Warmup (на отдельном стриме)
    with torch.no_grad():
        with torch.cuda.stream(stream):
            for _ in range(warmup_n):
                model(static_input)
        torch.cuda.synchronize()

    # Захват графа
    graph = torch.cuda.CUDAGraph()
    with torch.no_grad():
        with torch.cuda.stream(stream):
            torch.cuda.synchronize()
            graph.capture_begin()
            static_output.copy_(model(static_input))
            graph.capture_end()

    return graph, static_input, static_output

## Проверяем как работает функция записи CUDA graph

In [21]:
bsz, ch, height, width = 1, 3, 224, 224

graph, input_placeholder, output_placeholder = record_CUDA_graph(model, bsz, height, width, warmup_n=10)

test_data = torch.ones(bsz, ch, height, width, device="cuda")
# запускаем под no_grad, чтобы минимизировать потребление памяти (исключает выделение памяти под градиенты)
with torch.no_grad():
    # запускаем исходную модель
    model_output = model(test_data)
    
    # запускаем graph
    input_placeholder.copy_(test_data)
    graph.replay()
    graph_output = output_placeholder.clone()
    
    # сравниваем выходы
    assert torch.all(model_output == graph_output), "выход оригинальной модели и CUDA graph не совпадают"

## Сравниваем latency оригинальной модели и CUDA graph

In [26]:
def graph_runner(input_data):
    input_placeholder.copy_(input_data)
    graph.replay()
    return output_placeholder


torch.ones(1, 3, 224, 224, device='cuda')
# запускаем под no_grad, чтобы минимизировать потребление памяти (исключает выделение памяти под градиенты)
with torch.no_grad():
    latency_benchmark(
        model, 
        test_data, 
        warmup_n=10, 
        benchmark_n=10000,
    )
    latency_benchmark(
        graph_runner, 
        test_data, 
        warmup_n=10, 
        benchmark_n=10000,
    )

3.039ms +- 0.189ms
0.484ms +- 0.103ms


AssertionError: слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков

## Экспортируйте torchvision модель в onnx файл (10 баллов)

Для тестов нам потребуется 2 варианта onnx:
1. все входные оси фиксированны (1, 3, 224, 224). Файл назовите "my-model-ssss.onnx".
1. размер батча, высота и ширина динамические, количество входных каналов фиксированное (1, 3, height, width). Файл назовите "my-model-dsdd.onnx".

Имя входа должно быть "x", имя выхода "output".

Докуметацию по экспорту в onnx можно почитать [тут](https://pytorch.org/docs/stable/onnx_torchscript.html#torch.onnx.export).

In [27]:
model.cpu()
dummy_input = torch.randn(1, 3, 224, 224)
# 1)
torch.onnx.export(
    model,
    dummy_input,
    "my-model-ssss.onnx",
    input_names=["x"],
    output_names=["output"],
    dynamic_axes=None,
    opset_version=11
)

# 2)
torch.onnx.export(
    model,
    dummy_input,
    "my-model-dsdd.onnx",
    input_names=["x"],
    output_names=["output"],
    dynamic_axes={
        "x": {0: "batch", 2: "height", 3: "width"},
        "output": {0: "batch"} 
    },
    opset_version=11
)

## Скомпилируйте простейший вариант модели без динамических осей

In [28]:
from polygraphy.backend.trt import CreateConfig
from polygraphy.backend.trt import engine_from_network
from polygraphy.backend.trt import NetworkFromOnnxPath
from polygraphy.backend.trt import save_engine


model_ssss = NetworkFromOnnxPath("my-model-ssss.onnx")
config = CreateConfig()

engine = engine_from_network(model_ssss, config=config)
save_engine(engine, path="my-model-ssss.engine");

[W] 'colored' module is not installed, will not use colors when logging. To enable colors, please install the 'colored' module: python3 -m pip install colored
[I] Configuring with profiles:[
        Profile 0:
            {x [min=[1, 3, 224, 224], opt=[1, 3, 224, 224], max=[1, 3, 224, 224]]}
    ]
[I] Building engine with configuration:
    Flags                  | []
    Engine Capability      | EngineCapability.DEFAULT
    Memory Pools           | [WORKSPACE: 16080.25 MiB, TACTIC_DRAM: 16080.25 MiB]
    Tactic Sources         | [CUBLAS, CUBLAS_LT, CUDNN, EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [FASTER_DYNAMIC_SHAPES_0805, DISABLE_EXTERNAL_TACTIC_SOURCES_FOR_CORE_0805]
[I] Finished engine building in 19.525 seconds
[I] Saving engine to my-model-ssss.engine


## Проверти что точность не просела

**ВАЖНО:**<br>
Опция ``copy_outputs_to_host=False`` позволяет пропустить копирование данных с GPU на CPU.<br>
Вместо numpy array мы получим, torch.Tensor, что бывает очень полезно и экономит время на копировании.

In [29]:
from polygraphy.backend.trt import TrtRunner


with TrtRunner(engine) as trt_runner:
    def validation_trt_runner(input_data):
        # пропустим копирование на CPU copy_outputs_to_host=False
        output = trt_runner.infer(feed_dict={"x": input_data}, copy_outputs_to_host=False)
        return output['output']

    validate(validation_trt_runner, batch_size=1, height=224, width=224)

100%|██████████| 1557161267/1557161267 [01:14<00:00, 21021119.47it/s]


Extracting imagenette/imagenette2.tgz to imagenette
acc = 79.24%


## Сделайте замер latency с помощью ранее написанной функции latency_benchmark

In [32]:
with TrtRunner(engine) as trt_runner:
    def validation_trt_runner(input_data):
        # пропустим копирование на CPU copy_outputs_to_host=False
        output = trt_runner.infer(feed_dict={"x": input_data}, copy_outputs_to_host=False)
        return output['output']

    latency_benchmark(validation_trt_runner, test_input=torch.ones(1, 3, 224, 224), warmup_n=10, benchmark_n=10000)

1.172ms +- 1.625ms


AssertionError: слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков

## Теперь на основе примера выше скомпилируйте модель с динамическим batch size в диапазоне [1, 64] (5 баллов)
**ВАЖНО:**<br>
Как задать конфиг для динамических осей? Читаем доку [тут](https://docs.nvidia.com/deeplearning/tensorrt/polygraphy/docs/backend/trt/profile.html#optimization-profile) и добавляем профиль в config.

In [38]:
from polygraphy.backend.trt import Profile
model_dsdd = NetworkFromOnnxPath("my-model-dsdd.onnx")

profile = Profile()
profile.add("x", min=(1, 3, 224, 224), opt=(8, 3, 224, 224), max=(64, 3, 224, 224))

config = CreateConfig(profiles=[profile])
engine = engine_from_network(model_dsdd, config=config)
save_engine(engine, path="my-model-dsdd.engine")

[I] Configuring with profiles:[
        Profile 0:
            {x [min=(1, 3, 224, 224), opt=(8, 3, 224, 224), max=(64, 3, 224, 224)]}
    ]
[I] Building engine with configuration:
    Flags                  | []
    Engine Capability      | EngineCapability.DEFAULT
    Memory Pools           | [WORKSPACE: 16080.25 MiB, TACTIC_DRAM: 16080.25 MiB]
    Tactic Sources         | [CUBLAS, CUBLAS_LT, CUDNN, EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [FASTER_DYNAMIC_SHAPES_0805, DISABLE_EXTERNAL_TACTIC_SOURCES_FOR_CORE_0805]
[I] Finished engine building in 17.210 seconds
[I] Saving engine to my-model-dsdd.engine


## Проверте что точность не просела и замерте latency для batch size 1 и 64

In [39]:
with TrtRunner(engine) as trt_runner:
    def validation_trt_runner(input_data):
        output = trt_runner.infer(feed_dict={"x": input_data}, copy_outputs_to_host=False)
        return output['output']

    validate(validation_trt_runner, batch_size=1, height=224, width=224)
    latency_benchmark(validation_trt_runner, test_input=torch.ones(1, 3, 224, 224), warmup_n=10, benchmark_n=10000)

acc = 79.24%
1.343ms +- 1.890ms


AssertionError: слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков

In [40]:
with TrtRunner(engine) as trt_runner:
    def validation_trt_runner(input_data):
        output = trt_runner.infer(feed_dict={"x": input_data}, copy_outputs_to_host=False)
        return output['output']

    validate(validation_trt_runner, batch_size=64, height=224, width=224)
    latency_benchmark(validation_trt_runner, test_input=torch.ones(64, 3, 224, 224), warmup_n=10, benchmark_n=10000) # не вместилось видимо

acc = 79.24%
18.727ms +- 2.697ms


AssertionError: слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков

## Скомпилируйте квантованный вариант engine c динамическим batch size [1, 64] (15 балов)

Для этого вам потребуется на основе кода ``imagenette_val_dataloader`` сделать свой калибратор. Документацию на калибратор можно найти [тут](https://docs.nvidia.com/deeplearning/tensorrt/polygraphy/docs/backend/trt/calibrator.html#polygraphy.backend.trt.calibrator.Calibrator).

In [54]:
from polygraphy.backend.trt import Calibrator
def imagenette_calibration_data(dataloader):
    for images, labels in dataloader:
        images_np = np.ascontiguousarray(images.numpy()).astype(np.float32)        
        yield {"x": images_np}

val_dataloader = imagenette_val_dataloader(batch_size=8, height=224, width=224)
calibration_gen = imagenette_calibration_data(val_dataloader)
calibrator = Calibrator(
    data_loader=calibration_gen,
    cache="calibration.cache"
)

model_dsdd = NetworkFromOnnxPath("my-model-dsdd.onnx")

profile = Profile()
profile.add("x", min=(1,3,224,224), opt=(8,3,224,224), max=(64,3,224,224))

config = CreateConfig(
    profiles=[profile],
    int8=True,
    calibrator=calibrator
)

engine = engine_from_network(model_dsdd, config=config)
save_engine(engine, path="my-model-dsdd.int8.engine")

[I] Configuring with profiles:[
        Profile 0:
            {x [min=(1, 3, 224, 224), opt=(8, 3, 224, 224), max=(64, 3, 224, 224)]}
    ]
[I] Using calibration profile: {x [min=(1, 3, 224, 224), opt=(8, 3, 224, 224), max=(64, 3, 224, 224)]}
[I] Building engine with configuration:
    Flags                  | [INT8]
    Engine Capability      | EngineCapability.DEFAULT
    Memory Pools           | [WORKSPACE: 16080.25 MiB, TACTIC_DRAM: 16080.25 MiB]
    Tactic Sources         | [CUBLAS, CUBLAS_LT, CUDNN, EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [FASTER_DYNAMIC_SHAPES_0805, DISABLE_EXTERNAL_TACTIC_SOURCES_FOR_CORE_0805]
    Calibrator             | Calibrator(<generator object imagenette_calibration_data at 0x7d26797cc200>, cache='calibration.cache', BaseClass=<class 'tensorrt_bindings.tensorrt.IInt8EntropyCalibrator2'>)


[!] Input tensor: x | Received incompatible shape: (5, 3, 224, 224).
    Note: Expected a shape compatible with: BoundedShape([8, 3, 224, 224], min=None, max=None)


[I] Saving calibration cache to calibration.cache
[I] Finished engine building in 63.567 seconds
[I] Saving engine to my-model-dsdd.int8.engine


## Проверте что точность не просела и замерте latency для batch size 1 и 64

In [55]:
with TrtRunner(engine) as trt_runner:
    def validation_trt_runner(input_data):
        output = trt_runner.infer(feed_dict={"x": input_data}, copy_outputs_to_host=False)
        return output['output']

    validate(validation_trt_runner, batch_size=1, height=224, width=224)
    latency_benchmark(validation_trt_runner, test_input=torch.ones(1, 3, 224, 224), warmup_n=10, benchmark_n=10000)

acc = 77.53%
1.060ms +- 1.308ms


AssertionError: слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков

In [56]:
with TrtRunner(engine) as trt_runner:
    def validation_trt_runner(input_data):
        output = trt_runner.infer(feed_dict={"x": input_data}, copy_outputs_to_host=False)
        return output['output']

    validate(validation_trt_runner, batch_size=32, height=224, width=224)
    latency_benchmark(validation_trt_runner, test_input=torch.ones(32, 3, 224, 224), warmup_n=10, benchmark_n=10000)

acc = 77.53%
3.596ms +- 3.578ms


AssertionError: слишком большое отклонение в измерениях (> 10%), проверьте код, возможно стоит поднять кол-во запусков